# Palm Case Study

Tasks:
- Generate insights about ResortChain
- Identify main cash flow drivers (both for CRDIT an DBIT). 
- Outlier analysis, trends (e.g. spikes, seasonality, causality)

## 1) Load datasets

This section imports all provided datasets and prepares them for analysis. We start with `transactions.csv` (semicolon-separated, comma decimals) and also load `balances.csv`, `system_forecasts.csv`, and `user_forecasts.csv` for later use.


In [24]:
# CELL 1: SETUP & DATA LOADING
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Plotly defaults
px.defaults.template = 'plotly_white'
px.defaults.width = 1100
px.defaults.height = 500

# Configure dataset paths (absolute)
base_dir = Path("/Users/gianniskotsas/Documents/Side Projects/palm-case-study/scripts/datasets/raw")
transactions_path = base_dir / "transactions.csv"
balances_path = base_dir / "balances.csv"
system_forecasts_path = base_dir / "system_forecasts.csv"
user_forecasts_path = base_dir / "user_forecasts.csv"

# Helper: choose main date column name from available candidates
# IMPORTANT: value_date is prioritized over booking_date
DATE_CANDIDATES = [
    'value_date', 'booking_date', 'date', 'transaction_date', 'transactionDate'
]

def pick_date_column(df: pd.DataFrame) -> str | None:
    for c in DATE_CANDIDATES:
        if c in df.columns:
            return c
    # try to detect first column with datetime dtype
    for c in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[c]):
            return c
    # try to parse any column ending with 'date'
    for c in df.columns:
        if 'date' in c.lower():
            try:
                _ = pd.to_datetime(df[c], errors='raise')
                return c
            except Exception:
                continue
    return None

# Load Transactions: support CSV (expected) or Excel fallback
transactions = None
if transactions_path.suffix.lower() in {'.xlsx', '.xls'}:
    # Excel with potential multiple sheets
    xl = pd.ExcelFile(transactions_path)
    sheet_names = set([s.lower() for s in xl.sheet_names])
    try:
        tx_sheet = next(s for s in xl.sheet_names if s.lower() in {"transactions", "tx", "data"})
    except StopIteration:
        tx_sheet = xl.sheet_names[0]
    transactions = pd.read_excel(xl, sheet_name=tx_sheet)
else:
    # CSV (semicolon sep, comma decimal)
    parse_cols = ['value_date', 'booking_date']
    parse_present = [c for c in parse_cols if c in pd.read_csv(transactions_path, sep=';', nrows=0).columns]
    transactions = pd.read_csv(
        transactions_path,
        sep=';',
        decimal=',',
        dtype={
            'account_number': 'string',
            'currency': 'string',
            'credit_or_debit': 'string',
            'additional_info': 'string',
            'remittence_info': 'string',
            'bank_reference': 'string',
            'creditor_name': 'string',
            'debtor_name': 'string',
        },
        parse_dates=parse_present if len(parse_present) > 0 else None,
        dayfirst=True,
        infer_datetime_format=True,
        engine='python'
    )

# Load optional datasets when present
balances = None
if balances_path.exists():
    try:
        balances = pd.read_csv(balances_path, sep=';', decimal=',', parse_dates=['balance_date'])
    except Exception:
        balances = pd.read_csv(balances_path, sep=';', decimal=',')
        if 'balance_date' in balances.columns:
            balances['balance_date'] = pd.to_datetime(balances['balance_date'], errors='coerce')

system_forecasts = None
if system_forecasts_path.exists():
    system_forecasts = pd.read_csv(system_forecasts_path, sep=';', decimal=',')

user_forecasts = None
if user_forecasts_path.exists():
    user_forecasts = pd.read_csv(user_forecasts_path, sep=';', decimal=',')

# Identify main date column for transactions
trx_date_col = pick_date_column(transactions) if transactions is not None else None
if trx_date_col is not None:
    transactions[trx_date_col] = pd.to_datetime(transactions[trx_date_col], errors='coerce')

# Basic summary
print("=" * 60)
print("DATA LOADING SUMMARY")
print("=" * 60)
print(f"Transactions loaded: {transactions is not None}")
if transactions is not None:
    print(f"  Shape: {transactions.shape}")
    print(f"  📅 DATE COLUMN USED: {trx_date_col}")
print(f"\nBalances loaded: {balances is not None}")
if balances is not None:
    print(f"  Shape: {balances.shape}")
print(f"\nSystem forecasts loaded: {system_forecasts is not None}")
if system_forecasts is not None:
    print(f"  Shape: {system_forecasts.shape}")
print(f"\nUser forecasts loaded: {user_forecasts is not None}")
if user_forecasts is not None:
    print(f"  Shape: {user_forecasts.shape}")
print("=" * 60)

# Peek
display(transactions.head(5) if transactions is not None else "transactions missing")


DATA LOADING SUMMARY
Transactions loaded: True
  Shape: (1244, 11)
  📅 DATE COLUMN USED: value_date

Balances loaded: True
  Shape: (243, 6)

System forecasts loaded: True
  Shape: (108, 20)

User forecasts loaded: True
  Shape: (267, 18)


,account_number,value_date,booking_date,amount,currency,credit_or_debit,additional_info,remittence_info,bank_reference,creditor_name,debtor_name
0,FR7612345678901234567890123,2025-01-01,2025-01-01,525.98,EUR,CRDT,Resort Revenue - cash_in_resort_revenue,Transaction for Cash In Resort Revenue,REF343962,Resort Customer,France Account Account
1,FR7612345678901234567890123,2025-01-03,2025-01-01,198.51,EUR,DBIT,Investment Purchase - cash_out_investments_out...,Transaction for Cash Out Investments Outflow,REF308496,France Account Account,Investment Manager
2,FR7612345678901234567890123,2025-01-02,2025-01-01,151.97,EUR,DBIT,Investment Purchase - cash_out_investments_out...,Transaction for Cash Out Investments Outflow,REF717889,France Account Account,Investment Manager
3,FR7612345678901234567890123,2025-01-01,2025-01-01,426.18,EUR,DBIT,FX Transaction - cash_out_foreign_exchange_exp...,Transaction for Cash Out Foreign Exchange Expe...,REF895667,France Account Account,FX Bank
4,FR7612345678901234567890123,2025-01-02,2025-01-01,743.41,EUR,DBIT,Tax Payment - cash_out_tax_payments,Transaction for Cash Out Tax Payments,REF391369,France Account Account,Tax Authority


In [25]:
# OPTIONAL: OpenAI SDK Setup (if needed for future AI-powered analysis)
# Uncomment the code below if you want to use OpenAI:
#
# import os
# from openai import OpenAI
# from dotenv import load_dotenv
# 
# # Load .env.local from project root
# env_path = Path('/Users/gianniskotsas/Documents/Side Projects/palm-case-study/.env.local')
# if env_path.exists():
#     load_dotenv(env_path)
# 
# # Initialize OpenAI client
# openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
# print(f"OpenAI client initialized: {openai_client is not None}")

print("✓ Ready to create charts using loaded datasets")


✓ Ready to create charts using loaded datasets


## 2) Chart 1: Monthly Expenses by Category (Stacked Bar) + Hotel Revenue (Line)

This chart shows monthly expenses broken down by category as a stacked bar chart, with hotel revenue overlaid as a line chart.


In [38]:
# CHART 1: Monthly Expenses by Category (Stacked Bar) + Hotel Revenue (Line)

# Prepare data
tx = transactions.copy()
tx['month'] = pd.to_datetime(tx[trx_date_col]).dt.to_period('M')

# Extract category from additional_info
tx['category'] = tx['additional_info'].str.extract(r'(.*?) - ')[0]

# Separate expenses (DBIT) and revenue (CRDT)
expenses = tx[(tx['credit_or_debit'] == 'DBIT') & (tx['category'].notna())]
revenue = tx[(tx['credit_or_debit'] == 'CRDT')]

# Aggregate monthly expenses by category
monthly_expenses = expenses.groupby(['month', 'category'])['amount'].sum().reset_index()
monthly_expenses['month_str'] = monthly_expenses['month'].astype(str)

# Aggregate monthly revenue
monthly_revenue = revenue.groupby('month')['amount'].sum().reset_index()
monthly_revenue['month_str'] = monthly_revenue['month'].astype(str)

# Create stacked bar chart for expenses
fig1 = go.Figure()

# Add expense bars by category
categories = monthly_expenses['category'].unique()
for category in sorted(categories):
    cat_data = monthly_expenses[monthly_expenses['category'] == category]
    fig1.add_trace(go.Bar(
        x=cat_data['month_str'],
        y=cat_data['amount'],
        name=category,
        hovertemplate='%{x}<br>%{fullData.name}: €%{y:,.2f}<extra></extra>'
    ))

# Add revenue line
fig1.add_trace(go.Scatter(
    x=monthly_revenue['month_str'],
    y=monthly_revenue['amount'],
    name='Hotel Revenue',
    mode='lines+markers',
    line=dict(color='darkgreen', width=3),
    marker=dict(size=8),
    yaxis='y2',
    hovertemplate='%{x}<br>Revenue: €%{y:,.2f}<extra></extra>'
))

# Update layout
fig1.update_layout(
    title='Monthly Expenses by Category & Hotel Revenue',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Expenses (EUR)', side='left'),
    yaxis2=dict(title='Revenue (EUR)', overlaying='y', side='right'),
    barmode='stack',
    legend=dict(orientation='v', yanchor='top', y=1, xanchor='left', x=1.15),
    hovermode='x unified',
    height=600,
    width=1200
)

fig1.show()

# Calculate and display average percentage share of each expense category
print("\n=== Average Percentage Share of Expense Categories ===")
total_expenses_by_category = expenses.groupby('category')['amount'].sum().reset_index()
total_expenses_overall = total_expenses_by_category['amount'].sum()
total_expenses_by_category['percentage'] = (total_expenses_by_category['amount'] / total_expenses_overall) * 100
total_expenses_by_category = total_expenses_by_category.sort_values('percentage', ascending=False)

for idx, row in total_expenses_by_category.iterrows():
    print(f"{row['category']}: {row['percentage']:.2f}% (€{row['amount']:,.2f})")
print(f"\nTotal Expenses: €{total_expenses_overall:,.2f}")

# Create DataFrame for CSV export
# Pivot expenses to have categories as columns
monthly_expenses_pivot = monthly_expenses.pivot(index='month', columns='category', values='amount').reset_index()

# Rename columns to match expected format
category_mapping = {
    'Payroll Payment': 'Payroll',
    'Tax Payment': 'Tax',
    'FX Transaction': 'FX',
    'Investment Purchase': 'Investment',
    'Insurance Premium': 'Insurance',
    'Maintenance Expenses': 'Maintenance'
}
monthly_expenses_pivot = monthly_expenses_pivot.rename(columns=category_mapping)

# Add revenue column
monthly_revenue_for_merge = monthly_revenue.copy()
monthly_revenue_for_merge['month'] = monthly_revenue_for_merge['month'].astype(str).apply(lambda x: pd.Period(x, freq='M'))
monthly_expenses_pivot = monthly_expenses_pivot.merge(
    monthly_revenue_for_merge[['month', 'amount']], 
    on='month', 
    how='left'
).rename(columns={'amount': 'Revenue'})

# Convert month to date string
monthly_expenses_pivot['date'] = monthly_expenses_pivot['month'].astype(str)
monthly_expenses_pivot = monthly_expenses_pivot.drop('month', axis=1)

# Fill NaN with 0 and reorder columns
expected_columns = ['date', 'Maintenance', 'Insurance', 'Investment', 'FX', 'Tax', 'Payroll', 'Revenue']
for col in expected_columns:
    if col not in monthly_expenses_pivot.columns and col != 'date':
        monthly_expenses_pivot[col] = 0

monthly_expenses_pivot = monthly_expenses_pivot[expected_columns].fillna(0)

# Create alias for export
revenue_expenses_chart = monthly_expenses_pivot.copy()

print("\n✓ DataFrame 'revenue_expenses_chart' created for export")
print(f"Shape: {revenue_expenses_chart.shape}")
display(revenue_expenses_chart)

# Format output for JavaScript/TypeScript
print("\n" + "="*80)
print("COPY-PASTE READY FORMAT:")
print("="*80)
print("const data = [")

for idx, row in revenue_expenses_chart.iterrows():
    # Convert date string "2025-01" to "Jan 25" format
    month_period = pd.Period(row['date'], freq='M')
    date_formatted = month_period.strftime('%b %y')
    
    print(f"  {{ date: \"{date_formatted}\", Maintenance: {row['Maintenance']}, Insurance: {row['Insurance']}, Investment: {row['Investment']}, FX: {row['FX']}, Tax: {row['Tax']}, Payroll: {row['Payroll']}, Revenue: {row['Revenue']} }},")

print("];")
print("="*80)



=== Average Percentage Share of Expense Categories ===
Payroll Payment: 66.26% (€323,486.27)
Tax Payment: 16.06% (€78,415.76)
FX Transaction: 9.45% (€46,154.06)
Investment Purchase: 4.51% (€22,035.51)
Insurance Premium: 1.86% (€9,059.60)
Maintenance Expenses: 1.85% (€9,056.19)

Total Expenses: €488,207.39

✓ DataFrame 'revenue_expenses_chart' created for export
Shape: (8, 8)


,date,Maintenance,Insurance,Investment,FX,Tax,Payroll,Revenue
0,2025-01,1437.41,886.05,3904.26,7832.74,9771.65,32649.74,44279.36
1,2025-02,1570.15,1182.29,2769.50,4999.63,10052.17,29710.32,41297.05
2,2025-03,1017.61,1622.73,2932.03,6480.44,11745.89,49829.74,28979.67
3,2025-04,896.78,1015.38,814.60,4742.14,11405.08,52613.36,49437.74
4,2025-05,1077.97,1043.56,2282.34,6217.90,8674.07,40464.44,47120.29
5,2025-06,1257.97,1025.04,2231.32,5982.84,9607.59,41202.42,29327.26
6,2025-07,1170.00,953.69,3490.73,4275.71,6321.82,46288.00,27627.23
7,2025-08,628.30,1330.86,3610.73,5622.66,10837.49,30728.25,33330.01



COPY-PASTE READY FORMAT:
const data = [
  { date: "Jan 25", Maintenance: 1437.41, Insurance: 886.05, Investment: 3904.2599999999998, FX: 7832.74, Tax: 9771.65, Payroll: 32649.74, Revenue: 44279.36 },
  { date: "Feb 25", Maintenance: 1570.15, Insurance: 1182.29, Investment: 2769.5, FX: 4999.63, Tax: 10052.17, Payroll: 29710.32, Revenue: 41297.05 },
  { date: "Mar 25", Maintenance: 1017.61, Insurance: 1622.73, Investment: 2932.0299999999997, FX: 6480.44, Tax: 11745.89, Payroll: 49829.74, Revenue: 28979.670000000002 },
  { date: "Apr 25", Maintenance: 896.78, Insurance: 1015.38, Investment: 814.6, FX: 4742.14, Tax: 11405.08, Payroll: 52613.36, Revenue: 49437.74 },
  { date: "May 25", Maintenance: 1077.97, Insurance: 1043.56, Investment: 2282.34, FX: 6217.9, Tax: 8674.07, Payroll: 40464.44, Revenue: 47120.29 },
  { date: "Jun 25", Maintenance: 1257.97, Insurance: 1025.04, Investment: 2231.32, FX: 5982.84, Tax: 9607.59, Payroll: 41202.42, Revenue: 29327.260000000002 },
  { date: "Jul 25", 

In [37]:
import csv

# Export the revenue and expenses chart data to CSV using the DataFrame `revenue_expenses_chart`
# (If your DataFrame is named differently, update the variable name below accordingly.)

# Check if the DataFrame exists in the environment
if "revenue_expenses_chart" in locals():
    export_df = revenue_expenses_chart.copy()
elif "monthly_expenses_pivot" in locals():
    # Fallback: try to use monthly_expenses_pivot if available
    export_df = monthly_expenses_pivot.copy()
else:
    raise NameError("No DataFrame named 'revenue_expenses_chart' or 'monthly_expenses_pivot' found. Please define it before exporting.")

# Ensure the columns exist and are in the correct order
csv_columns = ["date", "Maintenance", "Insurance", "Investment", "FX", "Tax", "Payroll", "Revenue"]
missing_cols = [col for col in csv_columns if col not in export_df.columns]
if missing_cols:
    raise ValueError(f"Missing columns in export DataFrame: {missing_cols}")

# Reorder columns and convert to list of dicts
export_data = export_df[csv_columns].to_dict(orient="records")

csv_filename = "revenue_expenses_chart_data.csv"

with open(csv_filename, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for row in export_data:
        writer.writerow(row)

print(f"Chart data exported to {csv_filename}")


Chart data exported to revenue_expenses_chart_data.csv


In [ ]:
# CELL 2: DATA EXPLORATION & QUALITY ASSESSMENT
from typing import Optional

if transactions is None:
    display("Transactions dataset not found. Please ensure 'transactions.csv' exists under datasets/raw.")
else:
    # Helpers to detect amount column and coerce numerics
    AMOUNT_CANDIDATES = [
        'amount', 'booking_amount', 'transaction_amount', 'value', 'amt'
    ]

    def pick_amount_column(df: pd.DataFrame) -> Optional[str]:
        for c in AMOUNT_CANDIDATES:
            if c in df.columns:
                return c
        # guess: first numeric column named like amount
        for c in df.columns:
            if 'amount' in c.lower() or 'value' in c.lower():
                return c
        return None

    def coerce_amount(series: pd.Series) -> pd.Series:
        # If string with comma decimal, replace and convert
        if series.dtype == 'object' or pd.api.types.is_string_dtype(series):
            return pd.to_numeric(series.str.replace(',', '.', regex=False).str.replace(' ', ''), errors='coerce')
        return pd.to_numeric(series, errors='coerce')

    amount_col = pick_amount_column(transactions)
    if amount_col is None:
        # Try to infer from credits/debits
        amount_col = 'amount'
        transactions[amount_col] = pd.to_numeric(0)

    # Coerce date and amount columns
    if trx_date_col is not None:
        transactions[trx_date_col] = pd.to_datetime(transactions[trx_date_col], errors='coerce')
    transactions[amount_col] = coerce_amount(transactions[amount_col])

    # Preview head/tail
    display(transactions.head(5))
    display(transactions.tail(5))

    # Column names and dtypes
    display(pd.DataFrame({'dtype': transactions.dtypes.astype(str)}))

    # Missingness
    missing_counts = transactions.isna().sum().sort_values(ascending=False)
    missing_pct = (transactions.isna().mean() * 100).round(2)
    dq_missing = pd.concat([missing_counts.rename('missing_count'), missing_pct.rename('missing_%')], axis=1)
    display(dq_missing)

    # Date range
    date_min = transactions[trx_date_col].min() if trx_date_col else None
    date_max = transactions[trx_date_col].max() if trx_date_col else None
    print({'date_min': date_min, 'date_max': date_max})

    # Unique transaction categories/types if present
    category_like_cols = [c for c in transactions.columns if 'category' in c.lower() or 'type' in c.lower()]
    unique_info = {}
    for c in category_like_cols:
        nun = transactions[c].nunique(dropna=True)
        sample_vals = transactions[c].dropna().astype(str).value_counts().head(10)
        unique_info[c] = {'nunique': nun, 'top_values': sample_vals.to_dict()}
    display(unique_info)

    # Basic amount statistics
    amount_stats = transactions[amount_col].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]).to_frame('amount_stats')
    display(amount_stats)

    # Obvious data quality issues
    # 1) Duplicates
    dup_cols = [trx_date_col, amount_col, 'bank_reference'] if 'bank_reference' in transactions.columns else [trx_date_col, amount_col]
    potential_dups = transactions.duplicated(subset=[c for c in dup_cols if c is not None], keep=False).sum()

    # 2) Non-parsable dates or amounts
    non_parsable_dates = transactions[trx_date_col].isna().sum() if trx_date_col else None
    non_parsable_amounts = transactions[amount_col].isna().sum()

    # 3) Sign consistency vs credit_or_debit
    sign_inconsistencies = None
    if 'credit_or_debit' in transactions.columns:
        cod = transactions['credit_or_debit'].astype(str).str.upper()
        # infer sign from amount if signed; if all positive, we cannot check reliably
        if transactions[amount_col].notna().any():
            sign = np.sign(transactions[amount_col].fillna(0))
            sign_inconsistencies = ((cod == 'CRDT') & (sign < 0) | (cod.str.startswith(('DB', 'DR'))) & (sign > 0)).sum()

    dq_summary = pd.DataFrame([
        {'metric': 'rows', 'value': len(transactions)},
        {'metric': 'columns', 'value': transactions.shape[1]},
        {'metric': 'date_min', 'value': date_min},
        {'metric': 'date_max', 'value': date_max},
        {'metric': 'potential_duplicates', 'value': int(potential_dups)},
        {'metric': 'non_parsable_dates', 'value': None if non_parsable_dates is None else int(non_parsable_dates)},
        {'metric': 'non_parsable_amounts', 'value': int(non_parsable_amounts)},
        {'metric': 'sign_inconsistencies', 'value': None if sign_inconsistencies is None else int(sign_inconsistencies)},
    ])
    display(dq_summary)

    # Balances presence and date coverage
    if balances is not None:
        bmin = balances['balance_date'].min() if 'balance_date' in balances.columns else None
        bmax = balances['balance_date'].max() if 'balance_date' in balances.columns else None
        print({'balances_date_min': bmin, 'balances_date_max': bmax, 'balances_rows': len(balances)})

    # Forecast datasets presence
    print({'system_forecasts_present': system_forecasts is not None, 'user_forecasts_present': user_forecasts is not None})



,account_number,value_date,booking_date,amount,currency,credit_or_debit,additional_info,remittence_info,bank_reference,creditor_name,debtor_name
0,FR7612345678901234567890123,2025-01-01,2025-01-01,525.98,EUR,CRDT,Resort Revenue - cash_in_resort_revenue,Transaction for Cash In Resort Revenue,REF343962,Resort Customer,France Account Account
1,FR7612345678901234567890123,2025-01-03,2025-01-01,198.51,EUR,DBIT,Investment Purchase - cash_out_investments_out...,Transaction for Cash Out Investments Outflow,REF308496,France Account Account,Investment Manager
2,FR7612345678901234567890123,2025-01-02,2025-01-01,151.97,EUR,DBIT,Investment Purchase - cash_out_investments_out...,Transaction for Cash Out Investments Outflow,REF717889,France Account Account,Investment Manager
3,FR7612345678901234567890123,2025-01-01,2025-01-01,426.18,EUR,DBIT,FX Transaction - cash_out_foreign_exchange_exp...,Transaction for Cash Out Foreign Exchange Expe...,REF895667,France Account Account,FX Bank
4,FR7612345678901234567890123,2025-01-02,2025-01-01,743.41,EUR,DBIT,Tax Payment - cash_out_tax_payments,Transaction for Cash Out Tax Payments,REF391369,France Account Account,Tax Authority


,account_number,value_date,booking_date,amount,currency,credit_or_debit,additional_info,remittence_info,bank_reference,creditor_name,debtor_name
1239,FR7612345678901234567890123,2025-08-31,2025-08-30,79.07,EUR,DBIT,Maintenance Expenses - cash_out_resort_mainten...,Transaction for Cash Out Resort Maintenance Ex...,REF752004,France Account Account,Maintenance Co
1240,FR7612345678901234567890123,2025-08-31,2025-08-31,135.21,EUR,DBIT,Investment Purchase - cash_out_investments_out...,Transaction for Cash Out Investments Outflow,REF503953,France Account Account,Investment Manager
1241,FR7612345678901234567890123,2025-08-31,2025-08-31,98.99,EUR,DBIT,Maintenance Expenses - cash_out_resort_mainten...,Transaction for Cash Out Resort Maintenance Ex...,REF954141,France Account Account,Maintenance Co
1242,FR7612345678901234567890123,2025-08-31,2025-08-31,341.45,EUR,DBIT,FX Transaction - cash_out_foreign_exchange_exp...,Transaction for Cash Out Foreign Exchange Expe...,REF439566,France Account Account,FX Bank
1243,FR7612345678901234567890123,2025-08-31,2025-08-31,315.69,EUR,CRDT,Tax Refund - cash_in_tax_income,Transaction for Cash In Tax Income,REF357848,Government,France Account Account


,dtype
account_number,string
value_date,object
booking_date,datetime64[ns]
amount,float64
currency,string
credit_or_debit,string
additional_info,string
remittence_info,string
bank_reference,string
creditor_name,string


,missing_count,missing_%
account_number,0,0.0
value_date,0,0.0
booking_date,0,0.0
amount,0,0.0
currency,0,0.0
credit_or_debit,0,0.0
additional_info,0,0.0
remittence_info,0,0.0
bank_reference,0,0.0
creditor_name,0,0.0


{'date_min': Timestamp('2025-01-01 00:00:00'), 'date_max': Timestamp('2025-08-31 00:00:00')}


{}

,amount_stats
count,1244.000000
mean,634.731511
std,868.209700
min,17.100000
10%,58.914000
25%,104.012500
50%,276.615000
75%,681.632500
90%,2037.151000
max,3978.690000


,metric,value
0,rows,1244
1,columns,11
2,date_min,2025-01-01 00:00:00
3,date_max,2025-08-31 00:00:00
4,potential_duplicates,0
5,non_parsable_dates,0
6,non_parsable_amounts,0
7,sign_inconsistencies,839


{'balances_date_min': Timestamp('2025-01-01 00:00:00'), 'balances_date_max': Timestamp('2025-08-31 00:00:00'), 'balances_rows': 243}
{'system_forecasts_present': True, 'user_forecasts_present': True}


## 3) Chart 2: Revenue Heatmap (Day of Week vs Week of Year)

This heat map shows the hotel revenue pattern by day of the week and week of the year.


In [40]:
# CHART 2: Revenue Heatmap (Day of Week vs Week of Year)

# Confirm which date column is being used
print(f"Using date column for heatmap: {trx_date_col}")

# Prepare revenue data - use loaded transactions
revenue_data = transactions[(transactions['credit_or_debit'] == 'CRDT') & 
                             (transactions['additional_info'].str.contains('Resort Revenue', na=False))].copy()

revenue_data['date'] = pd.to_datetime(revenue_data[trx_date_col])
revenue_data['day_of_week'] = revenue_data['date'].dt.day_name()
# Convert week to integer properly
revenue_data['week_of_year'] = revenue_data['date'].dt.isocalendar().week.astype(int)

# Aggregate revenue by week and day
heatmap_data = revenue_data.groupby(['week_of_year', 'day_of_week'], as_index=False)['amount'].sum()

# Debug: check data
print(f"Heatmap data shape: {heatmap_data.shape}")
print(f"Sample data:\n{heatmap_data.head(10)}")
print(f"Week range: {heatmap_data['week_of_year'].min()} to {heatmap_data['week_of_year'].max()}")

# Pivot for heatmap
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_pivot = heatmap_data.pivot(index='day_of_week', columns='week_of_year', values='amount')
heatmap_pivot = heatmap_pivot.reindex(day_order)

# Replace NaN with 0 for visualization
heatmap_pivot = heatmap_pivot.fillna(0)

# Create heatmap
fig2 = go.Figure(data=go.Heatmap(
    z=heatmap_pivot.values,
    x=heatmap_pivot.columns.tolist(),
    y=heatmap_pivot.index.tolist(),
    colorscale='YlOrRd',
    hovertemplate='Week %{x}<br>%{y}<br>Revenue: €%{z:,.2f}<extra></extra>',
    colorbar=dict(title='Revenue (EUR)'),
    zmid=None  # Auto-scale colors
))

fig2.update_layout(
    title='Hotel Revenue Heatmap: Day of Week vs Week of Year',
    xaxis=dict(title='Week of Year', type='category'),
    yaxis=dict(title='Day of Week'),
    height=500,
    width=1200
)

fig2.show()


Using date column for heatmap: value_date
Heatmap data shape: (100, 3)
Sample data:
   week_of_year day_of_week   amount
0             1   Wednesday   525.98
1             2      Sunday  4905.90
2             2     Tuesday  2029.42
3             2   Wednesday  1939.65
4             3      Friday  1491.71
5             3      Monday  5634.42
6             3    Saturday  2998.90
7             4    Saturday   683.98
8             4    Thursday  2304.46
9             4     Tuesday  8578.78
Week range: 1 to 35


In [43]:
# Export heatmap data in copy-paste ready format for TypeScript/JavaScript

# Use the heatmap_data DataFrame from the previous cell
# Group by day and create the nested structure
day_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# First, let's analyze the data coverage
print("\n" + "="*80)
print("DATA COVERAGE ANALYSIS:")
print("="*80)

# Get the range of weeks in the data
all_weeks = sorted(heatmap_data['week_of_year'].unique())
min_week = heatmap_data['week_of_year'].min()
max_week = heatmap_data['week_of_year'].max()
print(f"Week range in data: {min_week} to {max_week}")
print(f"Total weeks with ANY data: {len(all_weeks)}")

# Create a complete grid of all possible day/week combinations
all_possible_weeks = range(min_week, max_week + 1)
total_possible_combinations = len(day_order) * len(all_possible_weeks)
actual_data_points = len(heatmap_data)

print(f"\nTotal possible day/week combinations: {total_possible_combinations}")
print(f"Actual data points with revenue > 0: {actual_data_points}")
print(f"Missing combinations: {total_possible_combinations - actual_data_points}")
print(f"Coverage: {(actual_data_points / total_possible_combinations * 100):.1f}%")

# Check each day of week
print("\n" + "-"*80)
print("BREAKDOWN BY DAY OF WEEK:")
print("-"*80)
for day in day_order:
    day_data = heatmap_data[heatmap_data['day_of_week'] == day]
    weeks_with_data = set(day_data['week_of_year'].values)
    missing_weeks = set(all_possible_weeks) - weeks_with_data
    
    print(f"\n{day}:")
    print(f"  Weeks with data: {len(weeks_with_data)} out of {len(all_possible_weeks)}")
    print(f"  Missing weeks: {len(missing_weeks)}")
    if len(missing_weeks) > 0 and len(missing_weeks) <= 10:
        print(f"  Missing week numbers: {sorted(missing_weeks)}")
    elif len(missing_weeks) > 10:
        print(f"  Missing week numbers: {sorted(list(missing_weeks)[:10])}... (showing first 10)")

# Now let's check the raw revenue data to see if missing means no transactions
print("\n" + "="*80)
print("CHECKING RAW TRANSACTION DATA:")
print("="*80)

# Go back to the original revenue_data to see all days
revenue_check = revenue_data.copy()
revenue_check['has_revenue'] = revenue_check['amount'] > 0

# Count by day of week
print("\nTransactions by day of week:")
for day in day_order:
    day_count = len(revenue_check[revenue_check['day_of_week'] == day])
    day_revenue = revenue_check[revenue_check['day_of_week'] == day]['amount'].sum()
    print(f"  {day}: {day_count} transactions, €{day_revenue:,.2f} total")

print("\n" + "="*80)
print("COPY-PASTE READY HEATMAP DATA (WITH COMPLETE WEEK COVERAGE):")
print("="*80)
print("const data = [")

for day in day_order:
    # Filter data for this day
    day_data = heatmap_data[heatmap_data['day_of_week'] == day].copy()
    
    # Create a dictionary for quick lookup of week -> revenue
    week_revenue_map = dict(zip(day_data['week_of_year'], day_data['amount']))
    
    # Format the data array - INCLUDING ALL WEEKS (missing ones get value: 0)
    data_entries = []
    for week in all_possible_weeks:
        # Check if this week has data, otherwise use 0
        value = round(week_revenue_map.get(week, 0), 2)
        data_entries.append(f'{{ weekOfYear: {week}, index: 1, value: {value} }}')
    
    # Join all entries for this day
    data_str = ', '.join(data_entries)
    
    print(f'  {{')
    print(f'    "name": "{day}",')
    print(f'    "data": [{data_str}]')
    print(f'  }},')

print("];")
print("="*80)
print(f"\nNote: Complete dataset includes ALL {len(all_possible_weeks)} weeks for each day.")
print(f"Missing week/day combinations now included with value: 0")

# Summary statistics
print(f"\nSummary:")
print(f"- Total data points: {len(heatmap_data)}")
print(f"- Days with data: {heatmap_data['day_of_week'].nunique()}")
print(f"- Weeks with data: {heatmap_data['week_of_year'].nunique()}")
print(f"- Total revenue: €{heatmap_data['amount'].sum():,.2f}")
print(f"- Average daily revenue: €{heatmap_data['amount'].mean():,.2f}")



DATA COVERAGE ANALYSIS:
Week range in data: 1 to 35
Total weeks with ANY data: 35

Total possible day/week combinations: 245
Actual data points with revenue > 0: 100
Missing combinations: 145
Coverage: 40.8%

--------------------------------------------------------------------------------
BREAKDOWN BY DAY OF WEEK:
--------------------------------------------------------------------------------

Sunday:
  Weeks with data: 13 out of 35
  Missing weeks: 22
  Missing week numbers: [1, 3, 4, 5, 7, 9, 13, 16, 17, 18]... (showing first 10)

Monday:
  Weeks with data: 16 out of 35
  Missing weeks: 19
  Missing week numbers: [1, 2, 4, 5, 10, 13, 15, 16, 20, 21]... (showing first 10)

Tuesday:
  Weeks with data: 15 out of 35
  Missing weeks: 20
  Missing week numbers: [1, 3, 6, 8, 11, 12, 14, 18, 19, 20]... (showing first 10)

Wednesday:
  Weeks with data: 19 out of 35
  Missing weeks: 16
  Missing week numbers: [3, 4, 5, 7, 8, 10, 12, 13, 32, 34]... (showing first 10)

Thursday:
  Weeks with d

## 4) Chart 3: Net Cashflow (Bar) + Cumulative Cash (Line)

This chart shows the daily net cashflow as bars and cumulative cash position as a line.


In [20]:
# CHART 3: Net Cashflow (Bar) + Cumulative Cash (Line)

# Get starting balance from balances.csv (first date)
starting_balance = balances.sort_values('balance_date')['amount'].iloc[0]
print(f"Starting balance from balances.csv: €{starting_balance:,.2f}")

# Calculate net cashflow per day - use loaded transactions
cashflow_data = transactions.copy()
cashflow_data['date'] = pd.to_datetime(cashflow_data[trx_date_col])
cashflow_data['amount_signed'] = cashflow_data.apply(
    lambda row: row['amount'] if row['credit_or_debit'] == 'CRDT' else -row['amount'], 
    axis=1
)

# Daily net cashflow
daily_cashflow = cashflow_data.groupby('date')['amount_signed'].sum().reset_index()
daily_cashflow.columns = ['date', 'net_cashflow']

# Calculate cumulative cash starting from the initial balance
daily_cashflow = daily_cashflow.sort_values('date')
daily_cashflow['cumulative_cash'] = starting_balance + daily_cashflow['net_cashflow'].cumsum()

# Create figure with secondary y-axis
fig3 = make_subplots(specs=[[{"secondary_y": True}]])

# Add net cashflow bars
fig3.add_trace(
    go.Bar(
        x=daily_cashflow['date'],
        y=daily_cashflow['net_cashflow'],
        name='Net Cashflow',
        marker_color=['green' if x >= 0 else 'red' for x in daily_cashflow['net_cashflow']],
        hovertemplate='%{x|%Y-%m-%d}<br>Net Cashflow: €%{y:,.2f}<extra></extra>'
    ),
    secondary_y=False
)

# Add cumulative cash line
fig3.add_trace(
    go.Scatter(
        x=daily_cashflow['date'],
        y=daily_cashflow['cumulative_cash'],
        name='Cumulative Cash',
        mode='lines',
        line=dict(color='blue', width=2),
        hovertemplate='%{x|%Y-%m-%d}<br>Cumulative: €%{y:,.2f}<extra></extra>'
    ),
    secondary_y=True
)

# Update layout
fig3.update_xaxes(title_text='Date')
fig3.update_yaxes(title_text='Net Cashflow (EUR)', secondary_y=False)
fig3.update_yaxes(title_text='Cumulative Cash (EUR)', secondary_y=True)

fig3.update_layout(
    title='Daily Net Cashflow & Cumulative Cash Position',
    hovermode='x unified',
    height=600,
    width=1200,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig3.show()


Starting balance from balances.csv: €614,655.83


## 5) Chart 4: Monthly Balance Deviations

This chart shows the deviation between actual end balances (from snapshots) and expected end balances (calculated from transactions using value_date).


In [21]:
# CHART 4: Monthly Balance Deviations

# Confirm which date column is being used
print(f"Using date column: {trx_date_col}")

# Prepare balances data - use loaded balances
bal = balances.copy()
bal['date'] = pd.to_datetime(bal['balance_date'])
bal['month'] = bal['date'].dt.to_period('M')

# Get first and last balance per month from snapshots
monthly_bal = bal.groupby('month').agg({
    'amount': ['first', 'last'],
    'date': ['min', 'max']
}).reset_index()
monthly_bal.columns = ['month', 'start_balance_snapshot', 'end_balance_snapshot', 'start_date', 'end_date']

# Calculate expected balances based on transactions - use loaded transactions
tx_calc = transactions.copy()
tx_calc['date'] = pd.to_datetime(tx_calc[trx_date_col])
tx_calc['month'] = tx_calc['date'].dt.to_period('M')
tx_calc['amount_signed'] = tx_calc.apply(
    lambda row: row['amount'] if row['credit_or_debit'] == 'CRDT' else -row['amount'], 
    axis=1
)

# Calculate monthly net change from transactions
monthly_tx_change = tx_calc.groupby('month')['amount_signed'].sum().reset_index()
monthly_tx_change.columns = ['month', 'tx_net_change']

# Merge with balance data
balance_comparison = monthly_bal.merge(monthly_tx_change, on='month', how='left')
balance_comparison['tx_net_change'] = balance_comparison['tx_net_change'].fillna(0)

# Calculate expected end balance
balance_comparison['expected_end_balance'] = balance_comparison['start_balance_snapshot'] + balance_comparison['tx_net_change']

# Calculate deviation
balance_comparison['deviation'] = balance_comparison['end_balance_snapshot'] - balance_comparison['expected_end_balance']
balance_comparison['month_str'] = balance_comparison['month'].astype(str)

# Create chart - only showing deviation
fig4 = go.Figure()

# Add deviation bars only
fig4.add_trace(go.Bar(
    x=balance_comparison['month_str'],
    y=balance_comparison['deviation'],
    name='Deviation',
    marker_color=['red' if x < 0 else 'green' for x in balance_comparison['deviation']],
    hovertemplate='%{x}<br>Deviation: €%{y:,.2f}<extra></extra>'
))

# Add a zero reference line
fig4.add_hline(y=0, line_dash="dash", line_color="gray", annotation_text="Zero Deviation")

fig4.update_layout(
    title='Monthly Balance Deviations: Actual vs Expected (from Transactions)',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Deviation (EUR)', zeroline=True),
    hovermode='x unified',
    height=600,
    width=1200,
    showlegend=False
)

fig4.show()

# Display deviation summary
print("\nBalance Deviation Summary:")
print(balance_comparison[['month_str', 'start_balance_snapshot', 'tx_net_change', 
                           'expected_end_balance', 'end_balance_snapshot', 'deviation']])


Using date column: booking_date



Balance Deviation Summary:
  month_str  start_balance_snapshot  tx_net_change  expected_end_balance  \
0   2025-01               614655.83      -12890.15             601765.68   
1   2025-02               601786.05       -8386.18             593399.87   
2   2025-03               592545.30      -46687.77             545857.53   
3   2025-04               545989.01      -20844.24             525144.77   
4   2025-05               525710.22      -15111.02             510599.20   
5   2025-06               513823.12      -29978.39             483844.73   
6   2025-07               480981.11      -31017.15             449963.96   
7   2025-08               448789.38      -21893.88             426895.50   

   end_balance_snapshot  deviation  
0             602353.54     587.86  
1             593366.53     -33.34  
2             548717.76    2860.23  
3             526668.16    1523.39  
4             514028.17    3428.97  
5             482048.25   -1796.48  
6             447175.53   -2

## 6) Chart 5: Daily Income Transactions with French Holidays

This chart shows daily income transactions as bars with vertical lines indicating French bank holidays.


In [44]:
# CHART 5: Daily Income Transactions with French Holidays

# Load bank holidays and filter for France
holidays_path = base_dir / "bank_holidays_2025.csv"
holidays_df = pd.read_csv(holidays_path)
france_holidays = holidays_df[holidays_df['Country'] == 'France'].copy()
france_holidays['Date'] = pd.to_datetime(france_holidays['Date'])

# Get daily income (CRDT transactions) - use loaded transactions
income_data = transactions[transactions['credit_or_debit'] == 'CRDT'].copy()
income_data['date'] = pd.to_datetime(income_data[trx_date_col])

# Aggregate daily income
daily_income = income_data.groupby('date')['amount'].sum().reset_index()
daily_income.columns = ['date', 'income']

# Create chart
fig5 = go.Figure()

# Add income bars
fig5.add_trace(go.Bar(
    x=daily_income['date'],
    y=daily_income['income'],
    name='Daily Income',
    marker_color='lightgreen',
    hovertemplate='%{x|%Y-%m-%d}<br>Income: €%{y:,.2f}<extra></extra>'
))

# Add vertical lines for French holidays using shapes
for idx, holiday in france_holidays.iterrows():
    # Convert Timestamp to datetime for plotly
    holiday_date = pd.Timestamp(holiday['Date']).to_pydatetime()
    
    fig5.add_shape(
        type="line",
        x0=holiday_date,
        x1=holiday_date,
        y0=0,
        y1=1,
        yref="paper",
        line=dict(color="red", width=2, dash="dash")
    )
    
    # Add annotation
    fig5.add_annotation(
        x=holiday_date,
        y=1,
        yref="paper",
        text=holiday['Holiday'],
        showarrow=False,
        textangle=-90,
        font=dict(size=8, color="red"),
        yanchor="bottom"
    )

fig5.update_layout(
    title='Daily Income Transactions with French Bank Holidays (2025)',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Income (EUR)'),
    hovermode='x unified',
    height=600,
    width=1200,
    showlegend=True
)

fig5.show()

# Create alias for export
daily_income_chart = daily_income.copy()
french_holidays_chart = france_holidays.copy()

print("\n✓ DataFrame 'daily_income_chart' created for export")
print(f"Shape: {daily_income_chart.shape}")
print(f"Date range: {daily_income_chart['date'].min()} to {daily_income_chart['date'].max()}")
print(f"Total income: €{daily_income_chart['income'].sum():,.2f}")

print("\n✓ DataFrame 'french_holidays_chart' created for export")
print(f"French holidays count: {len(french_holidays_chart)}")

# Format output for JavaScript/TypeScript
print("\n" + "="*80)
print("COPY-PASTE READY FORMAT (DAILY INCOME):")
print("="*80)
print("const dailyIncomeData = [")

for idx, row in daily_income_chart.iterrows():
    date_formatted = row['date'].strftime('%b %d')
    income_value = round(row['income'], 2)
    print(f"  {{ date: '{date_formatted}', income: {income_value} }},")

print("];")
print("="*80)

print("\n" + "="*80)
print("COPY-PASTE READY FORMAT (FRENCH HOLIDAYS):")
print("="*80)
print("const frenchHolidays = [")

for idx, row in french_holidays_chart.iterrows():
    date_formatted = row['Date'].strftime('%b %d')
    holiday_name = row['Holiday']
    print(f"  {{ date: '{date_formatted}', holiday: '{holiday_name}' }},")

print("];")
print("="*80)



✓ DataFrame 'daily_income_chart' created for export
Shape: (199, 2)
Date range: 2025-01-01 00:00:00 to 2025-08-31 00:00:00
Total income: €301,398.61

✓ DataFrame 'french_holidays_chart' created for export
French holidays count: 9

COPY-PASTE READY FORMAT (DAILY INCOME):
const dailyIncomeData = [
  { date: 'Jan 01', income: 525.98 },
  { date: 'Jan 03', income: 173.33 },
  { date: 'Jan 05', income: 204.59 },
  { date: 'Jan 06', income: 428.32 },
  { date: 'Jan 07', income: 2029.42 },
  { date: 'Jan 08', income: 2235.57 },
  { date: 'Jan 09', income: 275.75 },
  { date: 'Jan 10', income: 437.85 },
  { date: 'Jan 12', income: 5420.67 },
  { date: 'Jan 13', income: 5932.23 },
  { date: 'Jan 14', income: 287.61 },
  { date: 'Jan 15', income: 1282.59 },
  { date: 'Jan 16', income: 497.08 },
  { date: 'Jan 17', income: 2221.28 },
  { date: 'Jan 18', income: 2998.9 },
  { date: 'Jan 19', income: 411.89 },
  { date: 'Jan 21', income: 8969.09 },
  { date: 'Jan 22', income: 348.08 },
  { date: '

In [45]:
# Export Monthly Income Data (similar to the example format with categories)

# Create monthly aggregation of income
monthly_income_breakdown = income_data.copy()
monthly_income_breakdown['month'] = monthly_income_breakdown['date'].dt.to_period('M')

# Extract category/type from additional_info if available
monthly_income_breakdown['category'] = monthly_income_breakdown['additional_info'].str.extract(r'(.*?) - ')[0]

# If no clear categories, check for patterns in creditor_name or remittence_info
if monthly_income_breakdown['category'].isna().all():
    # Try to extract from creditor_name
    monthly_income_breakdown['category'] = 'Revenue'  # Default category
else:
    monthly_income_breakdown['category'] = monthly_income_breakdown['category'].fillna('Other')

# Group by month and category
monthly_income_by_category = monthly_income_breakdown.groupby(['month', 'category'])['amount'].sum().reset_index()

# Pivot to get categories as columns
monthly_income_pivot = monthly_income_by_category.pivot(index='month', columns='category', values='amount').reset_index()
monthly_income_pivot = monthly_income_pivot.fillna(0)

# Convert month to string
monthly_income_pivot['date'] = monthly_income_pivot['month'].astype(str)
monthly_income_pivot = monthly_income_pivot.drop('month', axis=1)

# Reorder columns
column_order = ['date'] + [col for col in monthly_income_pivot.columns if col != 'date']
monthly_income_pivot = monthly_income_pivot[column_order]

print("="*80)
print("MONTHLY INCOME BY CATEGORY:")
print("="*80)
display(monthly_income_pivot)

# Format for TypeScript/JavaScript (similar to the example provided)
print("\n" + "="*80)
print("COPY-PASTE READY FORMAT (MONTHLY INCOME - EXAMPLE STYLE):")
print("="*80)
print("const monthlyIncomeData = [")

for idx, row in monthly_income_pivot.iterrows():
    # Convert date string "2025-01" to "Jan 25" format
    month_period = pd.Period(row['date'], freq='M')
    date_formatted = month_period.strftime('%b %y')
    
    # Build the object dynamically based on available categories
    row_str = f"  {{ date: '{date_formatted}'"
    
    for col in monthly_income_pivot.columns:
        if col != 'date':
            value = round(row[col], 2)
            row_str += f", {col}: {value}"
    
    row_str += " },"
    print(row_str)

print("];")
print("="*80)

# Also provide a summary
print("\n" + "="*80)
print("SUMMARY:")
print("="*80)
print(f"Total months: {len(monthly_income_pivot)}")
print(f"Categories found: {[col for col in monthly_income_pivot.columns if col != 'date']}")
print(f"Total income: €{monthly_income_breakdown['amount'].sum():,.2f}")

# Create breakdown by category
if len(monthly_income_pivot.columns) > 2:  # More than just date + one category
    print("\nBreakdown by category:")
    for col in monthly_income_pivot.columns:
        if col != 'date':
            total = monthly_income_pivot[col].sum()
            pct = (total / monthly_income_breakdown['amount'].sum()) * 100
            print(f"  {col}: €{total:,.2f} ({pct:.1f}%)")


MONTHLY INCOME BY CATEGORY:


category,date,Investment Income,Resort Revenue,Tax Refund
0,2025-01,3367.12,33853.13,7059.11
1,2025-02,3103.59,32941.23,5252.23
2,2025-03,2644.63,18299.88,8035.16
3,2025-04,6276.67,38573.91,4587.16
4,2025-05,3852.89,39019.67,4247.73
5,2025-06,2840.91,19610.74,6875.61
6,2025-07,3088.56,17552.80,6985.87
7,2025-08,4311.61,22063.65,6954.75



COPY-PASTE READY FORMAT (MONTHLY INCOME - EXAMPLE STYLE):
const monthlyIncomeData = [
  { date: 'Jan 25', Investment Income: 3367.12, Resort Revenue: 33853.13, Tax Refund: 7059.11 },
  { date: 'Feb 25', Investment Income: 3103.59, Resort Revenue: 32941.23, Tax Refund: 5252.23 },
  { date: 'Mar 25', Investment Income: 2644.63, Resort Revenue: 18299.88, Tax Refund: 8035.16 },
  { date: 'Apr 25', Investment Income: 6276.67, Resort Revenue: 38573.91, Tax Refund: 4587.16 },
  { date: 'May 25', Investment Income: 3852.89, Resort Revenue: 39019.67, Tax Refund: 4247.73 },
  { date: 'Jun 25', Investment Income: 2840.91, Resort Revenue: 19610.74, Tax Refund: 6875.61 },
  { date: 'Jul 25', Investment Income: 3088.56, Resort Revenue: 17552.8, Tax Refund: 6985.87 },
  { date: 'Aug 25', Investment Income: 4311.61, Resort Revenue: 22063.65, Tax Refund: 6954.75 },
];

SUMMARY:
Total months: 8
Categories found: ['Investment Income', 'Resort Revenue', 'Tax Refund']
Total income: €301,398.61

Breakdown 

## 7) Chart 6: Investment Expenses & FX Fees

This chart shows investment expenses and FX fees as grouped bars for comparison.


In [29]:
# CHART 6: Exchange Fees Ratio (Line) + Investment Expenses (Bar)

# Prepare monthly data - use loaded transactions
tx_monthly = transactions.copy()
tx_monthly['date'] = pd.to_datetime(tx_monthly[trx_date_col])
tx_monthly['month'] = tx_monthly['date'].dt.to_period('M')

# Calculate monthly income (CRDT)
monthly_income = tx_monthly[tx_monthly['credit_or_debit'] == 'CRDT'].groupby('month')['amount'].sum().reset_index()
monthly_income.columns = ['month', 'income']

# Calculate monthly exchange fees (FX fees where debtor_name is 'FX Bank')
fx_expenses = tx_monthly[
    (tx_monthly['credit_or_debit'] == 'DBIT') & 
    (tx_monthly['debtor_name'] == 'FX Bank')
].groupby('month')['amount'].sum().reset_index()
fx_expenses.columns = ['month', 'fx_fees']

print(f"FX fees transactions found: {len(tx_monthly[(tx_monthly['credit_or_debit'] == 'DBIT') & (tx_monthly['debtor_name'] == 'FX Bank')])}")

# Calculate monthly investment expenses
investment_expenses = tx_monthly[
    (tx_monthly['credit_or_debit'] == 'DBIT') & 
    (tx_monthly['additional_info'].str.contains('Investment', na=False))
].groupby('month')['amount'].sum().reset_index()
investment_expenses.columns = ['month', 'investment_expenses']

# Merge data
chart6_data = monthly_income.merge(fx_expenses, on='month', how='left')
chart6_data = chart6_data.merge(investment_expenses, on='month', how='left')
chart6_data['fx_fees'] = chart6_data['fx_fees'].fillna(0)
chart6_data['investment_expenses'] = chart6_data['investment_expenses'].fillna(0)

chart6_data['month_str'] = chart6_data['month'].astype(str)

# Create grouped bar chart
fig6 = go.Figure()

# Add investment expenses bars
fig6.add_trace(go.Bar(
    x=chart6_data['month_str'],
    y=chart6_data['investment_expenses'],
    name='Investment Expenses',
    marker_color='lightcoral',
    hovertemplate='%{x}<br>Investment: €%{y:,.2f}<extra></extra>'
))

# Add FX fees bars
fig6.add_trace(go.Bar(
    x=chart6_data['month_str'],
    y=chart6_data['fx_fees'],
    name='FX Fees',
    marker_color='steelblue',
    hovertemplate='%{x}<br>FX Fees: €%{y:,.2f}<extra></extra>'
))

# Update layout for grouped bars
fig6.update_layout(
    title='Investment Expenses & FX Fees by Month',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Amount (EUR)'),
    barmode='group',
    hovermode='x unified',
    height=600,
    width=1200,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig6.show()

# Display summary stats
print("\nMonthly Investment & FX Fees Summary:")
print(chart6_data[['month_str', 'investment_expenses', 'fx_fees']])


FX fees transactions found: 153



Monthly Investment & FX Fees Summary:
  month_str  investment_expenses  fx_fees
0   2025-01              3904.26  7832.74
1   2025-02              2769.50  4999.63
2   2025-03              2932.03  6480.44
3   2025-04               814.60  4742.14
4   2025-05              2282.34  6217.90
5   2025-06              2231.32  5982.84
6   2025-07              3490.73  4275.71
7   2025-08              3610.73  5622.66


## Summary & Instructions

### Setup Instructions:

1. **Install Dependencies:**
   ```bash
   cd /Users/gianniskotsas/Documents/Side Projects/palm-case-study/scripts
   pip install -e .
   ```

2. **Run the Notebook:**
   - Execute cells in order
   - Cell 2: Load datasets (transactions, balances, forecasts)
   - Cell 3: Optional OpenAI setup (can skip)
   - Cells 4-15: Generate all 6 charts

### Charts Created:

1. **Monthly Expenses by Category + Revenue** - Stacked bar chart of expenses with hotel revenue line (includes average % share)
2. **Revenue Heatmap** - Day of week vs week of year
3. **Net Cashflow & Cumulative Cash** - Daily net cashflow bars with cumulative line (starting from initial balance in balances.csv)
4. **Monthly Balance Deviations** - Shows only deviation bars (using value_date) with color coding (green=positive, red=negative)
5. **Daily Income with French Holidays** - Income bars with French holiday markers
6. **Investment Expenses & FX Fees** - Grouped bar chart comparing investment expenses and FX fees

### Data Sources:
All charts use the pre-loaded datasets from Cell 2:
- `transactions` - Main transaction data
- `balances` - Daily balance snapshots
- `trx_date_col` - Auto-detected date column name
- `base_dir` - Path to datasets/raw folder
